In [1]:
!pip install -q duckdb --upgrade
!pip install  ducklake-delta-exporter
import sys
sys.exit(0)

sys.exit called with value 0. The interpreter will be restarted.


In [2]:
import duckdb
con = duckdb.connect()
con.sql(f""" 
                ATTACH or replace 'ducklake:/lakehouse/default/Files/meta.db' AS lake (DATA_PATH '/lakehouse/default/Tables') ;
                USE lake ;
                create schema if not exists aemo ; use aemo ; 
                CALL lake.set_option('parquet_row_group_size', 122880*20);
       """)

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

# **SQL**

In [3]:
Nbr_Files_to_Process  = 2500
source                = "/lakehouse/default/Files/Reports/Current/Daily_Reports/download_log.csv"
scada_today_path      = "/lakehouse/default/Files/Reports/Current/Dispatch_SCADA/download_log.csv"
price_today_path      = "/lakehouse/default/Files/Reports/Current/DispatchIS_Reports/download_log.csv"
output_path           = "/lakehouse/default/Files/excel/nem-registration-and-exemption-list_PU_and_Scheduled_Loads.csv"

In [4]:
scada_today =f"""

CREATE TABLE IF NOT EXISTS scada_today(
  DUID VARCHAR,
  INITIALMW DOUBLE,
  INTERVENTION DOUBLE,
  SETTLEMENTDATE TIMESTAMP WITH TIME ZONE,
  date DATE,
  file VARCHAR,
  PRIORITY INTEGER,
  "YEAR" BIGINT
);

SET VARIABLE list_of_files_scada_today = (
                                          WITH xxxx AS (
                                               SELECT
                                                concat('/lakehouse/default/Files/', extracted_filepath) AS file
                                                FROM '{scada_today_path}'
                                                WHERE parse_filename(extracted_filepath) NOT IN (SELECT DISTINCT file FROM scada_today)
                                                ORDER BY file
                                                LIMIT {Nbr_Files_to_Process}
                                                   )
                                           SELECT list(file) FROM xxxx 
  );

insert into scada_today by name
WITH raw AS (
  FROM read_csv(getvariable('list_of_files_scada_today'),
    Skip = 1,
    header = 0,
    all_varchar = 1,
    columns = {{
      'I' : 'VARCHAR',
      'DISPATCH' : 'VARCHAR',
      'UNIT_SCADA' : 'VARCHAR',
      'xx' : 'VARCHAR',
      'SETTLEMENTDATE' : 'timestamp',
      'DUID' : 'VARCHAR',
      'SCADAVALUE' : 'double',
      'LASTCHANGED' : 'timestamp'
    }},
    filename = 1,
    null_padding = true,
    ignore_errors = 1,
    auto_detect = false
  )
  WHERE I = 'D'
    AND SCADAVALUE != 0
)
SELECT
  DUID,
  SCADAVALUE AS INITIALMW,
  CAST(0 AS double) AS INTERVENTION,
  CAST(SETTLEMENTDATE AS TIMESTAMPTZ) AS SETTLEMENTDATE,
  CAST(SETTLEMENTDATE AS date) AS date,
  parse_filename(filename) AS file,
  0 AS PRIORITY,
  isoyear(CAST(SETTLEMENTDATE AS timestamp)) AS YEAR
FROM raw order by date

"""

In [5]:
price_today =f"""

CREATE TABLE IF NOT EXISTS  price_today(
  REGIONID VARCHAR,
  RUNNO DOUBLE,
  DISPATCHINTERVAL DOUBLE,
  INTERVENTION DOUBLE,
  RRP DOUBLE,
  EEP DOUBLE,
  ROP DOUBLE,
  APCFLAG DOUBLE,
  MARKETSUSPENDEDFLAG DOUBLE,
  RAISE6SECRRP DOUBLE,
  RAISE6SECROP DOUBLE,
  RAISE6SECAPCFLAG DOUBLE,
  RAISE60SECRRP DOUBLE,
  RAISE60SECROP DOUBLE,
  RAISE60SECAPCFLAG DOUBLE,
  RAISE5MINRRP DOUBLE,
  RAISE5MINROP DOUBLE,
  RAISE5MINAPCFLAG DOUBLE,
  RAISEREGRRP DOUBLE,
  RAISEREGROP DOUBLE,
  RAISEREGAPCFLAG DOUBLE,
  LOWER6SECRRP DOUBLE,
  LOWER6SECROP DOUBLE,
  LOWER6SECAPCFLAG DOUBLE,
  LOWER60SECRRP DOUBLE,
  LOWER60SECROP DOUBLE,
  LOWER60SECAPCFLAG DOUBLE,
  LOWER5MINRRP DOUBLE,
  LOWER5MINROP DOUBLE,
  LOWER5MINAPCFLAG DOUBLE,
  LOWERREGRRP DOUBLE,
  LOWERREGROP DOUBLE,
  LOWERREGAPCFLAG DOUBLE,
  PRE_AP_ENERGY_PRICE DOUBLE,
  PRE_AP_RAISE6_PRICE DOUBLE,
  PRE_AP_RAISE60_PRICE DOUBLE,
  PRE_AP_RAISE5MIN_PRICE DOUBLE,
  PRE_AP_RAISEREG_PRICE DOUBLE,
  PRE_AP_LOWER6_PRICE DOUBLE,
  PRE_AP_LOWER60_PRICE DOUBLE,
  PRE_AP_LOWER5MIN_PRICE DOUBLE,
  PRE_AP_LOWERREG_PRICE DOUBLE,
  RAISE1SECRRP DOUBLE,
  RAISE1SECROP DOUBLE,
  RAISE1SECAPCFLAG DOUBLE,
  LOWER1SECRRP DOUBLE,
  LOWER1SECROP DOUBLE,
  LOWER1SECAPCFLAG DOUBLE,
  PRE_AP_RAISE1_PRICE DOUBLE,
  PRE_AP_LOWER1_PRICE DOUBLE,
  CUMUL_PRE_AP_ENERGY_PRICE DOUBLE,
  CUMUL_PRE_AP_RAISE6_PRICE DOUBLE,
  CUMUL_PRE_AP_RAISE60_PRICE DOUBLE,
  CUMUL_PRE_AP_RAISE5MIN_PRICE DOUBLE,
  CUMUL_PRE_AP_RAISEREG_PRICE DOUBLE,
  CUMUL_PRE_AP_LOWER6_PRICE DOUBLE,
  CUMUL_PRE_AP_LOWER60_PRICE DOUBLE,
  CUMUL_PRE_AP_LOWER5MIN_PRICE DOUBLE,
  CUMUL_PRE_AP_LOWERREG_PRICE DOUBLE,
  CUMUL_PRE_AP_RAISE1_PRICE DOUBLE,
  CUMUL_PRE_AP_LOWER1_PRICE DOUBLE,
  "week" DOUBLE,
  SETTLEMENTDATE TIMESTAMP WITH TIME ZONE,
  date DATE,
  file VARCHAR,
  "YEAR" BIGINT
);

SET VARIABLE list_of_files_price_today = (
                                            WITH xxxx AS (
                                               SELECT
                                                concat('/lakehouse/default/Files/', extracted_filepath) AS file
                                                FROM '{price_today_path}'
                                                WHERE parse_filename(extracted_filepath) NOT IN (SELECT DISTINCT file FROM price_today)
                                                ORDER BY file
                                                LIMIT {Nbr_Files_to_Process}
                                                   )
                                             SELECT list(file) FROM xxxx );



insert into price_today by name
WITH RAW_price AS (
    FROM read_csv(getvariable('list_of_files_price_today'),
      Skip = 1,
      header = 0,
      all_varchar = 1,
      columns = {{
        'I' : 'VARCHAR',
        'DISPATCH' : 'VARCHAR',
        'PRICE' : 'VARCHAR',
        'xx' : 'VARCHAR',
        'SETTLEMENTDATE' : 'VARCHAR',
        'RUNNO' : 'VARCHAR',
        'REGIONID' : 'VARCHAR',
        'DISPATCHINTERVAL' : 'VARCHAR',
        'INTERVENTION' : 'VARCHAR',
        'RRP' : 'VARCHAR',
        'EEP' : 'VARCHAR',
        'ROP' : 'VARCHAR',
        'APCFLAG' : 'VARCHAR',
        'MARKETSUSPENDEDFLAG' : 'VARCHAR',
        'LASTCHANGED' : 'VARCHAR',
        'RAISE6SECRRP' : 'VARCHAR',
        'RAISE6SECROP' : 'VARCHAR',
        'RAISE6SECAPCFLAG' : 'VARCHAR',
        'RAISE60SECRRP' : 'VARCHAR',
        'RAISE60SECROP' : 'VARCHAR',
        'RAISE60SECAPCFLAG' : 'VARCHAR',
        'RAISE5MINRRP' : 'VARCHAR',
        'RAISE5MINROP' : 'VARCHAR',
        'RAISE5MINAPCFLAG' : 'VARCHAR',
        'RAISEREGRRP' : 'VARCHAR',
        'RAISEREGROP' : 'VARCHAR',
        'RAISEREGAPCFLAG' : 'VARCHAR',
        'LOWER6SECRRP' : 'VARCHAR',
        'LOWER6SECROP' : 'VARCHAR',
        'LOWER6SECAPCFLAG' : 'VARCHAR',
        'LOWER60SECRRP' : 'VARCHAR',
        'LOWER60SECROP' : 'VARCHAR',
        'LOWER60SECAPCFLAG' : 'VARCHAR',
        'LOWER5MINRRP' : 'VARCHAR',
        'LOWER5MINROP' : 'VARCHAR',
        'LOWER5MINAPCFLAG' : 'VARCHAR',
        'LOWERREGRRP' : 'VARCHAR',
        'LOWERREGROP' : 'VARCHAR',
        'LOWERREGAPCFLAG' : 'VARCHAR',
        'PRICE_STATUS' : 'VARCHAR',
        'PRE_AP_ENERGY_PRICE' : 'VARCHAR',
        'PRE_AP_RAISE6_PRICE' : 'VARCHAR',
        'PRE_AP_RAISE60_PRICE' : 'VARCHAR',
        'PRE_AP_RAISE5MIN_PRICE' : 'VARCHAR',
        'PRE_AP_RAISEREG_PRICE' : 'VARCHAR',
        'PRE_AP_LOWER6_PRICE' : 'VARCHAR',
        'PRE_AP_LOWER60_PRICE' : 'VARCHAR',
        'PRE_AP_LOWER5MIN_PRICE' : 'VARCHAR',
        'PRE_AP_LOWERREG_PRICE' : 'VARCHAR',
        'RAISE1SECRRP' : 'VARCHAR',
        'RAISE1SECROP' : 'VARCHAR',
        'RAISE1SECAPCFLAG' : 'VARCHAR',
        'LOWER1SECRRP' : 'VARCHAR',
        'LOWER1SECROP' : 'VARCHAR',
        'LOWER1SECAPCFLAG' : 'VARCHAR',
        'PRE_AP_RAISE1_PRICE' : 'VARCHAR',
        'PRE_AP_LOWER1_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_ENERGY_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_RAISE6_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_RAISE60_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_RAISE5MIN_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_RAISEREG_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_LOWER6_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_LOWER60_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_LOWER5MIN_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_LOWERREG_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_RAISE1_PRICE' : 'VARCHAR',
        'CUMUL_PRE_AP_LOWER1_PRICE' : 'VARCHAR',
        'OCD_STATUS' : 'VARCHAR',
        'MII_STATUS' : 'VARCHAR',
      }},
      filename = 1,
      null_padding = true,
      ignore_errors = 1,
      auto_detect = false
    )
    WHERE I = 'D'
      AND PRICE = 'PRICE'
  )
  SELECT
    REGIONID,
    CAST(columns(* EXCLUDE (SETTLEMENTDATE, REGIONID, I,xx, PRICE, filename, OCD_STATUS, MII_STATUS, DISPATCH, PRICE_STATUS, LASTCHANGED)) AS DOUBLE), -- Cast remaining columns to DOUBLE
    CAST(SETTLEMENTDATE AS TIMESTAMPTZ) AS SETTLEMENTDATE,
    CAST(SETTLEMENTDATE AS date) AS date,
    parse_filename(filename) AS file,
    year(CAST(SETTLEMENTDATE AS TIMESTAMPTZ)) AS YEAR
  FROM RAW_price
  ORDER BY date

"""

In [6]:
scada = f"""

CREATE TABLE IF NOT EXISTS scada (
    UNIT VARCHAR, DUID VARCHAR, VERSION DOUBLE,
    RUNNO DOUBLE, INTERVENTION DOUBLE, DISPATCHMODE DOUBLE,
    AGCSTATUS DOUBLE, INITIALMW DOUBLE, TOTALCLEARED DOUBLE,
    RAMPDOWNRATE DOUBLE, RAMPUPRATE DOUBLE, LOWER5MIN DOUBLE,
    LOWER60SEC DOUBLE, LOWER6SEC DOUBLE, RAISE5MIN DOUBLE,
    RAISE60SEC DOUBLE, RAISE6SEC DOUBLE, MARGINAL5MINVALUE DOUBLE,
    MARGINAL60SECVALUE DOUBLE, MARGINAL6SECVALUE DOUBLE, MARGINALVALUE DOUBLE,
    VIOLATION5MINDEGREE DOUBLE, VIOLATION60SECDEGREE DOUBLE, VIOLATION6SECDEGREE DOUBLE,
    VIOLATIONDEGREE DOUBLE, LOWERREG DOUBLE, RAISEREG DOUBLE,
    AVAILABILITY DOUBLE, RAISE6SECFLAGS DOUBLE, RAISE60SECFLAGS DOUBLE,
    RAISE5MINFLAGS DOUBLE, RAISEREGFLAGS DOUBLE, LOWER6SECFLAGS DOUBLE,
    LOWER60SECFLAGS DOUBLE, LOWER5MINFLAGS DOUBLE, LOWERREGFLAGS DOUBLE,
    RAISEREGAVAILABILITY DOUBLE, RAISEREGENABLEMENTMAX DOUBLE, RAISEREGENABLEMENTMIN DOUBLE,
    LOWERREGAVAILABILITY DOUBLE, LOWERREGENABLEMENTMAX DOUBLE, LOWERREGENABLEMENTMIN DOUBLE,
    RAISE6SECACTUALAVAILABILITY DOUBLE, RAISE60SECACTUALAVAILABILITY DOUBLE, RAISE5MINACTUALAVAILABILITY DOUBLE,
    RAISEREGACTUALAVAILABILITY DOUBLE, LOWER6SECACTUALAVAILABILITY DOUBLE, LOWER60SECACTUALAVAILABILITY DOUBLE,
    LOWER5MINACTUALAVAILABILITY DOUBLE, LOWERREGACTUALAVAILABILITY DOUBLE,
    file VARCHAR, SETTLEMENTDATE TIMESTAMP, DATE DATE,
    YEAR SMALLINT
);

SET VARIABLE list_of_files_scada = (
                                            WITH xxxx AS (
                                               SELECT
                                                concat('/lakehouse/default/Files/', extracted_filepath) AS file
                                                FROM '{source}'
                                                WHERE parse_filename(extracted_filepath) NOT IN (SELECT DISTINCT file FROM scada)
                                                ORDER BY file
                                                LIMIT {Nbr_Files_to_Process}
                                                   )
                                                SELECT list(file) FROM xxxx );
insert into scada by name
WITH raw AS (
    FROM read_csv(
        getvariable('list_of_files_scada'),
        skip = 1,
        header = 0,
        all_varchar = 1,
        hive_partitioning = false ,
        columns = {{
            'I': 'VARCHAR', 'UNIT': 'VARCHAR', 'XX': 'VARCHAR', 'VERSION': 'VARCHAR',
            'SETTLEMENTDATE': 'VARCHAR', 'RUNNO': 'VARCHAR', 'DUID': 'VARCHAR', 'INTERVENTION': 'VARCHAR',
            'DISPATCHMODE': 'VARCHAR', 'AGCSTATUS': 'VARCHAR', 'INITIALMW': 'VARCHAR', 'TOTALCLEARED': 'VARCHAR',
            'RAMPDOWNRATE': 'VARCHAR', 'RAMPUPRATE': 'VARCHAR', 'LOWER5MIN': 'VARCHAR', 'LOWER60SEC': 'VARCHAR',
            'LOWER6SEC': 'VARCHAR', 'RAISE5MIN': 'VARCHAR', 'RAISE60SEC': 'VARCHAR', 'RAISE6SEC': 'VARCHAR',
            'MARGINAL5MINVALUE': 'VARCHAR', 'MARGINAL60SECVALUE': 'VARCHAR', 'MARGINAL6SECVALUE': 'VARCHAR', 'MARGINALVALUE': 'VARCHAR',
            'VIOLATION5MINDEGREE': 'VARCHAR', 'VIOLATION60SECDEGREE': 'VARCHAR', 'VIOLATION6SECDEGREE': 'VARCHAR', 'VIOLATIONDEGREE': 'VARCHAR',
            'LOWERREG': 'VARCHAR', 'RAISEREG': 'VARCHAR', 'AVAILABILITY': 'VARCHAR', 'RAISE6SECFLAGS': 'VARCHAR',
            'RAISE60SECFLAGS': 'VARCHAR', 'RAISE5MINFLAGS': 'VARCHAR', 'RAISEREGFLAGS': 'VARCHAR', 'LOWER6SECFLAGS': 'VARCHAR',
            'LOWER60SECFLAGS': 'VARCHAR', 'LOWER5MINFLAGS': 'VARCHAR', 'LOWERREGFLAGS': 'VARCHAR', 'RAISEREGAVAILABILITY': 'VARCHAR',
            'RAISEREGENABLEMENTMAX': 'VARCHAR', 'RAISEREGENABLEMENTMIN': 'VARCHAR', 'LOWERREGAVAILABILITY': 'VARCHAR', 'LOWERREGENABLEMENTMAX': 'VARCHAR',
            'LOWERREGENABLEMENTMIN': 'VARCHAR', 'RAISE6SECACTUALAVAILABILITY': 'VARCHAR', 'RAISE60SECACTUALAVAILABILITY': 'VARCHAR', 'RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
            'RAISEREGACTUALAVAILABILITY': 'VARCHAR', 'LOWER6SECACTUALAVAILABILITY': 'VARCHAR', 'LOWER60SECACTUALAVAILABILITY': 'VARCHAR', 'LOWER5MINACTUALAVAILABILITY': 'VARCHAR',
            'LOWERREGACTUALAVAILABILITY': 'VARCHAR'
        }},
        filename = 1,
        null_padding = true,
        ignore_errors = 1,
        auto_detect = false
    )
    WHERE
        I = 'D'
        AND UNIT = 'DUNIT'
        AND VERSION = 3
)
SELECT
    UNIT,
    DUID,
    CAST(columns(* EXCLUDE (DUID, UNIT, SETTLEMENTDATE, I, XX, filename)) AS DOUBLE),
    parse_filename(filename) AS file,
    CAST(SETTLEMENTDATE AS TIMESTAMP) AS SETTLEMENTDATE,
    CAST(SETTLEMENTDATE AS DATE) AS DATE,
    CAST(YEAR(CAST(SETTLEMENTDATE AS TIMESTAMP)) AS INT16) AS YEAR
FROM raw
"""

In [7]:
price =f"""
CREATE TABLE IF NOT EXISTS price (
    UNIT VARCHAR, REGIONID VARCHAR, VERSION DOUBLE,
    RUNNO DOUBLE, INTERVENTION DOUBLE, RRP DOUBLE,
    EEP DOUBLE, ROP DOUBLE, APCFLAG DOUBLE,
    MARKETSUSPENDEDFLAG DOUBLE, TOTALDEMAND DOUBLE, DEMANDFORECAST DOUBLE,
    DISPATCHABLEGENERATION DOUBLE, DISPATCHABLELOAD DOUBLE, NETINTERCHANGE DOUBLE,
    EXCESSGENERATION DOUBLE, LOWER5MINDISPATCH DOUBLE, LOWER5MINIMPORT DOUBLE,
    LOWER5MINLOCALDISPATCH DOUBLE, LOWER5MINLOCALPRICE DOUBLE, LOWER5MINLOCALREQ DOUBLE,
    LOWER5MINPRICE DOUBLE, LOWER5MINREQ DOUBLE, LOWER5MINSUPPLYPRICE DOUBLE,
    LOWER60SECDISPATCH DOUBLE, LOWER60SECIMPORT DOUBLE, LOWER60SECLOCALDISPATCH DOUBLE,
    LOWER60SECLOCALPRICE DOUBLE, LOWER60SECLOCALREQ DOUBLE, LOWER60SECPRICE DOUBLE,
    LOWER60SECREQ DOUBLE, LOWER60SECSUPPLYPRICE DOUBLE, LOWER6SECDISPATCH DOUBLE,
    LOWER6SECIMPORT DOUBLE, LOWER6SECLOCALDISPATCH DOUBLE, LOWER6SECLOCALPRICE DOUBLE,
    LOWER6SECLOCALREQ DOUBLE, LOWER6SECPRICE DOUBLE, LOWER6SECREQ DOUBLE,
    LOWER6SECSUPPLYPRICE DOUBLE, RAISE5MINDISPATCH DOUBLE, RAISE5MINIMPORT DOUBLE,
    RAISE5MINLOCALDISPATCH DOUBLE, RAISE5MINLOCALPRICE DOUBLE, RAISE5MINLOCALREQ DOUBLE,
    RAISE5MINPRICE DOUBLE, RAISE5MINREQ DOUBLE, RAISE5MINSUPPLYPRICE DOUBLE,
    RAISE60SECDISPATCH DOUBLE, RAISE60SECIMPORT DOUBLE, RAISE60SECLOCALDISPATCH DOUBLE,
    RAISE60SECLOCALPRICE DOUBLE, RAISE60SECLOCALREQ DOUBLE, RAISE60SECPRICE DOUBLE,
    RAISE60SECREQ DOUBLE, RAISE60SECSUPPLYPRICE DOUBLE, RAISE6SECDISPATCH DOUBLE,
    RAISE6SECIMPORT DOUBLE, RAISE6SECLOCALDISPATCH DOUBLE, RAISE6SECLOCALPRICE DOUBLE,
    RAISE6SECLOCALREQ DOUBLE, RAISE6SECPRICE DOUBLE, RAISE6SECREQ DOUBLE,
    RAISE6SECSUPPLYPRICE DOUBLE, AGGREGATEDISPATCHERROR DOUBLE, AVAILABLEGENERATION DOUBLE,
    AVAILABLELOAD DOUBLE, INITIALSUPPLY DOUBLE, CLEAREDSUPPLY DOUBLE,
    LOWERREGIMPORT DOUBLE, LOWERREGLOCALDISPATCH DOUBLE, LOWERREGLOCALREQ DOUBLE,
    LOWERREGREQ DOUBLE, RAISEREGIMPORT DOUBLE, RAISEREGLOCALDISPATCH DOUBLE,
    RAISEREGLOCALREQ DOUBLE, RAISEREGREQ DOUBLE, RAISE5MINLOCALVIOLATION DOUBLE,
    RAISEREGLOCALVIOLATION DOUBLE, RAISE60SECLOCALVIOLATION DOUBLE, RAISE6SECLOCALVIOLATION DOUBLE,
    LOWER5MINLOCALVIOLATION DOUBLE, LOWERREGLOCALVIOLATION DOUBLE, LOWER60SECLOCALVIOLATION DOUBLE,
    LOWER6SECLOCALVIOLATION DOUBLE, RAISE5MINVIOLATION DOUBLE, RAISEREGVIOLATION DOUBLE,
    RAISE60SECVIOLATION DOUBLE, RAISE6SECVIOLATION DOUBLE, LOWER5MINVIOLATION DOUBLE,
    LOWERREGVIOLATION DOUBLE, LOWER60SECVIOLATION DOUBLE, LOWER6SECVIOLATION DOUBLE,
    RAISE6SECRRP DOUBLE, RAISE6SECROP DOUBLE, RAISE6SECAPCFLAG DOUBLE,
    RAISE60SECRRP DOUBLE, RAISE60SECROP DOUBLE, RAISE60SECAPCFLAG DOUBLE,
    RAISE5MINRRP DOUBLE, RAISE5MINROP DOUBLE, RAISE5MINAPCFLAG DOUBLE,
    RAISEREGRRP DOUBLE, RAISEREGROP DOUBLE, RAISEREGAPCFLAG DOUBLE,
    LOWER6SECRRP DOUBLE, LOWER6SECROP DOUBLE, LOWER6SECAPCFLAG DOUBLE,
    LOWER60SECRRP DOUBLE, LOWER60SECROP DOUBLE, LOWER60SECAPCFLAG DOUBLE,
    LOWER5MINRRP DOUBLE, LOWER5MINROP DOUBLE, LOWER5MINAPCFLAG DOUBLE,
    LOWERREGRRP DOUBLE, LOWERREGROP DOUBLE, LOWERREGAPCFLAG DOUBLE,
    RAISE6SECACTUALAVAILABILITY DOUBLE, RAISE60SECACTUALAVAILABILITY DOUBLE, RAISE5MINACTUALAVAILABILITY DOUBLE,
    RAISEREGACTUALAVAILABILITY DOUBLE, LOWER6SECACTUALAVAILABILITY DOUBLE, LOWER60SECACTUALAVAILABILITY DOUBLE,
    LOWER5MINACTUALAVAILABILITY DOUBLE, LOWERREGACTUALAVAILABILITY DOUBLE, LORSURPLUS DOUBLE,
    LRCSURPLUS DOUBLE, file VARCHAR, SETTLEMENTDATE TIMESTAMP,
    DATE DATE, YEAR SMALLINT
);


SET VARIABLE list_of_files_price = (
                                            WITH xxxx AS (
                                               SELECT
                                                concat('/lakehouse/default/Files/', extracted_filepath) AS file
                                                FROM '{source}'
                                                WHERE parse_filename(extracted_filepath) NOT IN (SELECT DISTINCT file FROM price)
                                                ORDER BY file
                                                LIMIT {Nbr_Files_to_Process}
                                                   )
                                                SELECT list(file) FROM xxxx );


                                            
insert into price BY NAME
WITH raw AS (
    FROM read_csv(
        getvariable('list_of_files_price'),
        skip = 1,
        header = 0,
        all_varchar = 1,
        hive_partitioning = false ,
  columns={{
  'I': 'VARCHAR','UNIT': 'VARCHAR','XX': 'VARCHAR','VERSION': 'VARCHAR','SETTLEMENTDATE': 'VARCHAR','RUNNO': 'VARCHAR',
  'REGIONID': 'VARCHAR','INTERVENTION': 'VARCHAR','RRP': 'VARCHAR','EEP': 'VARCHAR','ROP': 'VARCHAR','APCFLAG': 'VARCHAR',
  'MARKETSUSPENDEDFLAG': 'VARCHAR','TOTALDEMAND': 'VARCHAR','DEMANDFORECAST': 'VARCHAR','DISPATCHABLEGENERATION': 'VARCHAR',
  'DISPATCHABLELOAD': 'VARCHAR','NETINTERCHANGE': 'VARCHAR','EXCESSGENERATION': 'VARCHAR','LOWER5MINDISPATCH': 'VARCHAR',
  'LOWER5MINIMPORT': 'VARCHAR','LOWER5MINLOCALDISPATCH': 'VARCHAR','LOWER5MINLOCALPRICE': 'VARCHAR','LOWER5MINLOCALREQ': 'VARCHAR',
  'LOWER5MINPRICE': 'VARCHAR','LOWER5MINREQ': 'VARCHAR','LOWER5MINSUPPLYPRICE': 'VARCHAR','LOWER60SECDISPATCH': 'VARCHAR','LOWER60SECIMPORT': 'VARCHAR',
  'LOWER60SECLOCALDISPATCH': 'VARCHAR','LOWER60SECLOCALPRICE': 'VARCHAR','LOWER60SECLOCALREQ': 'VARCHAR','LOWER60SECPRICE': 'VARCHAR',
  'LOWER60SECREQ': 'VARCHAR','LOWER60SECSUPPLYPRICE': 'VARCHAR','LOWER6SECDISPATCH': 'VARCHAR','LOWER6SECIMPORT': 'VARCHAR',
  'LOWER6SECLOCALDISPATCH': 'VARCHAR','LOWER6SECLOCALPRICE': 'VARCHAR','LOWER6SECLOCALREQ': 'VARCHAR','LOWER6SECPRICE': 'VARCHAR',
  'LOWER6SECREQ': 'VARCHAR','LOWER6SECSUPPLYPRICE': 'VARCHAR','RAISE5MINDISPATCH': 'VARCHAR','RAISE5MINIMPORT': 'VARCHAR',
  'RAISE5MINLOCALDISPATCH': 'VARCHAR','RAISE5MINLOCALPRICE': 'VARCHAR','RAISE5MINLOCALREQ': 'VARCHAR','RAISE5MINPRICE': 'VARCHAR',
  'RAISE5MINREQ': 'VARCHAR','RAISE5MINSUPPLYPRICE': 'VARCHAR','RAISE60SECDISPATCH': 'VARCHAR','RAISE60SECIMPORT': 'VARCHAR',
  'RAISE60SECLOCALDISPATCH': 'VARCHAR','RAISE60SECLOCALPRICE': 'VARCHAR','RAISE60SECLOCALREQ': 'VARCHAR','RAISE60SECPRICE': 'VARCHAR',
  'RAISE60SECREQ': 'VARCHAR','RAISE60SECSUPPLYPRICE': 'VARCHAR','RAISE6SECDISPATCH': 'VARCHAR','RAISE6SECIMPORT': 'VARCHAR',
  'RAISE6SECLOCALDISPATCH': 'VARCHAR','RAISE6SECLOCALPRICE': 'VARCHAR','RAISE6SECLOCALREQ': 'VARCHAR','RAISE6SECPRICE': 'VARCHAR',
  'RAISE6SECREQ': 'VARCHAR','RAISE6SECSUPPLYPRICE': 'VARCHAR','AGGREGATEDISPATCHERROR': 'VARCHAR','AVAILABLEGENERATION': 'VARCHAR',
  'AVAILABLELOAD': 'VARCHAR','INITIALSUPPLY': 'VARCHAR','CLEAREDSUPPLY': 'VARCHAR','LOWERREGIMPORT': 'VARCHAR','LOWERREGLOCALDISPATCH': 'VARCHAR',
  'LOWERREGLOCALREQ': 'VARCHAR','LOWERREGREQ': 'VARCHAR','RAISEREGIMPORT': 'VARCHAR','RAISEREGLOCALDISPATCH': 'VARCHAR','RAISEREGLOCALREQ': 'VARCHAR',
  'RAISEREGREQ': 'VARCHAR','RAISE5MINLOCALVIOLATION': 'VARCHAR','RAISEREGLOCALVIOLATION': 'VARCHAR','RAISE60SECLOCALVIOLATION': 'VARCHAR',
  'RAISE6SECLOCALVIOLATION': 'VARCHAR','LOWER5MINLOCALVIOLATION': 'VARCHAR','LOWERREGLOCALVIOLATION': 'VARCHAR','LOWER60SECLOCALVIOLATION': 'VARCHAR',
  'LOWER6SECLOCALVIOLATION': 'VARCHAR','RAISE5MINVIOLATION': 'VARCHAR','RAISEREGVIOLATION': 'VARCHAR','RAISE60SECVIOLATION': 'VARCHAR',
  'RAISE6SECVIOLATION': 'VARCHAR','LOWER5MINVIOLATION': 'VARCHAR','LOWERREGVIOLATION': 'VARCHAR','LOWER60SECVIOLATION': 'VARCHAR',
  'LOWER6SECVIOLATION': 'VARCHAR','RAISE6SECRRP': 'VARCHAR','RAISE6SECROP': 'VARCHAR','RAISE6SECAPCFLAG': 'VARCHAR','RAISE60SECRRP': 'VARCHAR',
  'RAISE60SECROP': 'VARCHAR','RAISE60SECAPCFLAG': 'VARCHAR','RAISE5MINRRP': 'VARCHAR','RAISE5MINROP': 'VARCHAR','RAISE5MINAPCFLAG': 'VARCHAR',
  'RAISEREGRRP': 'VARCHAR','RAISEREGROP': 'VARCHAR','RAISEREGAPCFLAG': 'VARCHAR','LOWER6SECRRP': 'VARCHAR','LOWER6SECROP': 'VARCHAR',
  'LOWER6SECAPCFLAG': 'VARCHAR','LOWER60SECRRP': 'VARCHAR','LOWER60SECROP': 'VARCHAR','LOWER60SECAPCFLAG': 'VARCHAR','LOWER5MINRRP': 'VARCHAR',
  'LOWER5MINROP': 'VARCHAR','LOWER5MINAPCFLAG': 'VARCHAR','LOWERREGRRP': 'VARCHAR','LOWERREGROP': 'VARCHAR','LOWERREGAPCFLAG': 'VARCHAR',
  'RAISE6SECACTUALAVAILABILITY': 'VARCHAR','RAISE60SECACTUALAVAILABILITY': 'VARCHAR','RAISE5MINACTUALAVAILABILITY': 'VARCHAR',
  'RAISEREGACTUALAVAILABILITY': 'VARCHAR','LOWER6SECACTUALAVAILABILITY': 'VARCHAR','LOWER60SECACTUALAVAILABILITY': 'VARCHAR',
  'LOWER5MINACTUALAVAILABILITY': 'VARCHAR','LOWERREGACTUALAVAILABILITY': 'VARCHAR','LORSURPLUS': 'VARCHAR','LRCSURPLUS': 'VARCHAR',
  }},
  filename =1,null_padding = true,ignore_errors=1,auto_detect=false)
  where I='D' and UNIT ='DREGION' AND VERSION = 3
                  )
    select
    UNIT,
    REGIONID,
    cast(columns(*exclude(REGIONID,UNIT,SETTLEMENTDATE,I,XX,filename)) as double),
    parse_filename(filename) as file,
    cast (SETTLEMENTDATE as TIMESTAMP) as SETTLEMENTDATE,
    cast(SETTLEMENTDATE as date) as DATE,
	  cast(year (cast (SETTLEMENTDATE as timestamp)) AS INT16) as YEAR
    from raw
      """

In [8]:
calendar ="""
            create table if not exists calendar as 
            SELECT
            cast(unnest(generate_series(cast ('2018-04-01' as date), cast('2026-12-31' as date), interval 1 day)) as date) as date,
            EXTRACT(year from date) as year,
            EXTRACT(month from date) as month
            """

In [9]:
mstdatetime ="""
create table if not exists mstdatetime as 
WITH data AS (
    SELECT
        UNNEST(GENERATE_SERIES('2023-01-01 00:00:00'::TIMESTAMPTZ, '2023-01-01 23:55:00'::TIMESTAMPTZ, INTERVAL 5 MINUTE)) AS interval_time
)
SELECT
    CAST(STRFTIME(interval_time, '%H%M') AS INT16) AS time,
    CAST(STRFTIME(interval_time, '%H') AS INT16) AS hour
FROM data;
           """

In [10]:
duid =f"""
create or replace table duid as


WITH
  duid_aemo AS (
    SELECT
      DUID AS DUID,
      first(Region) AS Region,
      first("Fuel Source - Descriptor") AS FuelSourceDescriptor,
      first(Participant) AS Participant
    FROM '{output_path}',
    
    WHERE
      length(DUID) > 2
    GROUP BY
      DUID
  ),
  states AS (
    SELECT 'WA1' AS RegionID, 'Western Australia' AS States
    UNION ALL SELECT 'QLD1', 'Queensland'
    UNION ALL SELECT 'NSW1', 'New South Walles'
    UNION ALL SELECT 'TAS1', 'Tasmania'
    UNION ALL SELECT 'SA1', 'South Australia'
    UNION ALL SELECT 'VIC1', 'Victoria'
  ),
  x AS (
    SELECT
      'WA1' AS Region,
      "Facility Code" AS DUID,
      "Participant Name" AS Participant
    FROM read_csv_auto('https://data.wa.aemo.com.au/datafiles/post-facilities/facilities.csv')
  ),
  tt AS (
    SELECT
      *
    FROM read_csv_auto('https://github.com/djouallah/aemo_fabric/raw/main/WA_ENERGY.csv', header = 1)
  ),
  duid_wa AS (
    SELECT
      x.DUID,
      x.Region,
      Technology AS FuelSourceDescriptor,
      x.Participant
    FROM x
    LEFT JOIN tt ON x.DUID = tt.DUID
  ),
  duid_all AS (
    SELECT
      *
    FROM duid_aemo
    UNION ALL
    SELECT
      *
    FROM duid_wa
  ),
geo as(
  select duid, max(latitude) as latitude,max(longitude) as longitude from
  read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR_I3U-f2DtY4ex8QXV_S1T19JYy58__nz52Ra6Mm10r3_vJik5OrvQecN-pFWfjUbIE6m0wMl_R6kL/pub?gid=0&single=true&output=csv')

  where latitude is not null
group by all)
SELECT
  a.DUID,
  Region,
  UPPER(LEFT(TRIM(FuelSourceDescriptor), 1)) || LOWER(SUBSTR(TRIM(FuelSourceDescriptor), 2)) AS FuelSourceDescriptor,
  Participant,
  states.States AS State,
  geo.latitude,
  geo.longitude
FROM duid_all a
JOIN states ON a.Region = states.RegionID
left JOIN geo ON a.duid = geo.duid;
"""

In [11]:
summary_incremental = f"""
CREATE TABLE IF NOT EXISTS summary(date DATE,"time" SMALLINT, cutoff TIMESTAMP WITH TIME ZONE, DUID VARCHAR,  mw DECIMAL(18,4), price DECIMAL(18,4));
SET VARIABLE max_timestamp = (SELECT max(cutoff) from summary );
insert into summary BY NAME
with incremental as (
    SELECT
        s.date,
        s.SETTLEMENTDATE,
        s.DUID,
        max(s.INITIALMW)  AS mw,
        max(p.RRP)  AS price
    FROM scada_today s
    JOIN duid d ON s.DUID = d.DUID
    JOIN (SELECT * FROM price_today WHERE INTERVENTION = 0  and date  >= cast(getvariable('max_timestamp') as date)) p
        ON s.SETTLEMENTDATE = p.SETTLEMENTDATE AND d.Region = p.REGIONID
    WHERE
        s.INTERVENTION = 0
        AND INITIALMW <> 0
        AND p.INTERVENTION = 0
        AND s.date           >= cast(getvariable('max_timestamp') as date)
        AND s.settlementdate > getvariable('max_timestamp')
        AND p.settlementdate > getvariable('max_timestamp')
    GROUP BY ALL)
    SELECT
    date,
    CAST(strftime(SETTLEMENTDATE, '%H%M') AS INT16) AS time,
    DUID,
    mw,
    price,
    MAX(SETTLEMENTDATE) OVER () AS cutoff
FROM incremental
ORDER BY date, DUID, time, price;
"""

In [12]:
summary_backfill = """
        create or replace table summary as
        select
        s.DATE as date,
        cast(strftime(s.SETTLEMENTDATE, '%H%M') AS INT16)                       as time ,
        (select max(cast(settlementdate as TIMESTAMPTZ) ) from scada)           as cutoff ,
        s.DUID,
        max(s.INITIALMW)                                                        as mw,
        max(p.RRP)                                                              as price
      from  scada   s
            LEFT JOIN duid d    ON s.DUID = d.DUID
            LEFT JOIN price   p ON s.SETTLEMENTDATE = p.SETTLEMENTDATE AND d.Region = p.REGIONID
      where
        s.INTERVENTION = 0
        and INITIALMW <> 0
        and p.INTERVENTION = 0
        --and s.YEAR = 2020
      group by all
      ORDER BY date, s.DUID, time, price;
      """

# **Run Queries**

In [13]:
%%time
from ducklake_delta_exporter import generate_latest_delta_log
try:
    con.sql(price)
    con.sql(scada)
    con.sql(duid)
    con.sql(calendar)
    con.sql(mstdatetime)
    con.sql(summary_backfill)
    generate_latest_delta_log('/lakehouse/default/Files/meta.db')
    ##################### maintenance #######################
    con.sql(f""" 
                CALL ducklake_expire_snapshots ('lake', older_than => now() - INTERVAL '3 day') ;
                CALL ducklake_cleanup_old_files('lake', older_than => now() - INTERVAL '3 day') ;
        """)
except Exception as e:
    print(e)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Parser Error: read_csv cannot take NULL list as parameter
CPU times: user 162 ms, sys: 34.1 ms, total: 196 ms
Wall time: 4.95 s


In [14]:
%%time
try:
    con.sql(f""" CALL lake.merge_adjacent_files() ; BEGIN TRANSACTION;  {scada_today} ; {price_today} ; {summary_incremental} ;  COMMIT ;  """)
    generate_latest_delta_log('/lakehouse/default/Files/meta.db')
except Exception as e:
    print(e)
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ aemo.calendar: Delta table already at latest DuckLake snapshot 2 (Delta version 1), skipping export.
✅ aemo.mstdatetime: Delta table already at latest DuckLake snapshot 3 (Delta version 1), skipping export.
✅ aemo.price: Delta table already at latest DuckLake snapshot 9723 (Delta version 21), skipping export.
✅ aemo.scada: Delta table already at latest DuckLake snapshot 9724 (Delta version 23), skipping export.
✅ aemo.scada_today: Delta log written v4411 (DuckLake snapshot: 9878)
📸 aemo.scada_today: Checkpoint created at Delta version 4411 (DuckLake snapshot: 9878)
🧹 aemo.scada_today: Cleaning up old log and checkpoint files before Delta version 4411...
  Deleted JSON log: 00000000000000004410.json
  Deleted checkpoint: 00000000000000001778.checkpoint.parquet
  Deleted JSON log: 00000000000000001778.json
  Deleted checkpoint: 00000000000000001780.checkpoint.parquet
  Deleted JSON log: 00000000000000001780.json
  Deleted checkpoint: 00000000000000001782.checkpoint.parquet
  Deleted JS